# Using Physics-Informed Neural Network to Solve the 1D Heat Equation

In this simple tutorial, we will solve the one-dimensional heat equation using a Physics-Informed Neural Network (PINN) implemented in TensorFlow. The problem definition can be interpreted as **cooling a one-dimensional rod** from an initial temperature distribution $u(x,0)=f(x)$. 

The problem will be treated as a **forward problem**, i.e. we will define the initial condition (IC) and boundary conditions (BC), as well as the governing differential equation, to approximate the (unique) solution function $u(x,t)$ using a PINN.

The governing partial differential equation (PDE) with the corresponding initial (IC) and boundary (BC) condition are given in non-dimensionalized form by:

- **PDE:** $\frac{\partial u(x,t)}{\partial t} - \frac{\partial^2 u(x,t)}{\partial x^2} = 0$
- **IC:** $u(x,0) = f(x), \quad 0 < x < 1$
- **BC:** $u(0,t) = u(1,t) = 0, \quad t > 0$

Let's start by loading the necessary modules.


In [ ]:
import numpy as np
import tensorflow as tf

## Defining Initial Condition and Analytical Solution

For this tutorial, we use $f(x)=sin(\pi x)$ as an initial temperature distribution, since the solution for this problem is given analytically $\left(u(x,t)=sin(\pi x)\cdot exp(-\pi^2 t)\right)$ and we can thus easily evaluate the performance of the PINN afterwards.


In [ ]:
# Defining the initial temperate distribution
f = lambda x: np.sin(np.pi*x)

# Analytical solution
def get_solution(X):
    """
    Computes the analytical solution (only valid for this particular problem setup).

    Parameters:
    - X: An array of spatio-temporal coordinates (x,t).

    Returns:
    - u_sol: The analytical solution for the temperature at the specified coordinates.
    """
    x = X[:,0]
    t = X[:,1]
    u_solution = f(x) * np.exp(-np.pi**2*t)
    # Expand to 2d-array (needed for PINN training)
    return np.expand_dims(u_solution, axis=1)

## Defining the Loss Functions

Next, we construct the **Physics Loss** $L_F$ that encodes the PDE, as well as the **Data Loss** $L_D$ that will enforce the IC and BC. For the physics loss, we use TensorFlow's `tf.GradientTape()` to obtain the partial derivatives $\frac{\partial u_\theta}{\partial t}$ and $\frac{\partial^2 u_\theta}{\partial x^2}$.

In [ ]:
def physics_loss(model, X_collocation):
    """
    Compute the physics-based loss for the PINN model.

    Parameters:
    - model: The PINN model which approximates the solution u(x,t).
    - X_col: A tensor containing the collocation points (x, t) where the PDE is enforced.

    Returns:
    - loss_physics: The mean squared residual of the PDE at the collocation points.
    """
    # First GradientTape for first-order derivative
    with tf.GradientTape(persistent=True) as tape1:
        tape1.watch(X_collocation)
        # Second GradientTape for second-order derivative
        with tf.GradientTape(persistent=True) as tape2:
            tape2.watch(X_collocation)
            u_pred = model(X_collocation)
        # First derivative w.r.t position
        u_x = tape2.gradient(u_pred, X_collocation)[:, 0]
    # First derivative w.r.t. time
    u_t = tape1.gradient(u_pred, X_collocation)[:, 1]
    # Second derivative w.r.t. position
    u_xx = tape1.gradient(u_x, X_collocation)[:, 0]

    # Physics residuals as given by the differential equation
    f_res = u_t - u_xx
    # Mean squared error loss for the physics residuals
    return tf.reduce_mean(tf.square(f_res))

def data_loss(model, X_train, u_train):
    """
    Compute the data loss for the PINN model.

    Parameters:
    - model: The PINN model which approximates the solution u(x,t).
    - X_train: A tensor containing the initial/boundary points (x, t).
    - u_train: A tensor containing the known values of u at the initial/boundary points.

    Returns:
    - loss_data: The mean squared error between the predicted and known values of u at the initial/boundary points.
    """
    u_pred = model(X_train)
    # Mean squared error loss for the data residuals
    return tf.reduce_mean(tf.square(u_pred - u_train))

## Defining Sampling Functions for Data Points

To train the PINN, we need to create datasets for the IC, BC, and collocation points where the PDE is enforced. We generate these points using the given problem definition (see above).

We implement an auxilary function, that randomly samples data points from the **computational domain $x\in[0,1]$ and $t\in[0,0.5]$**. This function will be later on used to sample collocation points for the physics loss function, as well as datasets for the test set and IC/BC (that will be further specified in the later part). The standard sampling approach for PINN is **Latin hypercube sampling** (lhs) to generate a near-random sample from a multidimensional distribution.

In [ ]:
from pyDOE import lhs

def sample_domain(n_sample):
    """
    Generate random data sampled from the computational domain.

    Parameter:
    - n_sample: An integer defining the number of data points sampled from the computational domain

    Returns:
    - X: The coordinates of the sampled data points
    """
    return [1, 0.5] * lhs(2, n_sample)

Now we create a random input dataset $\{x^{(i)},0\}_{i=1}^{N_{IC}}$ for the IC, with the corresponding temperature values. **Note: $x\in[0,1]$ while $t=0$**.

In [ ]:
def sample_IC(n_IC):
    """
    Generate random sample of the IC points (x, 0) and their corresponding temperature values

    Parameter:
    - n_IC: An integer defining the number of data points sampled from the initial conditions

    Returns:
    - X_IC: The coordinates of the initial conditions points
    - u_IC: The temperature values for the initial conditions points
    """
    X_IC = sample_domain(n_IC)
    # Set time to zero (x, 0)
    X_IC[:, 1] = 0
    # Get temperature from the initial condition (expand to 2d-array)
    u_IC = np.expand_dims(f(X_IC[:, 0]), axis=1)
    return X_IC, u_IC

For the BC, we randomly sample time values and construct the input datasets $\{0, t^{(i)}\}_{i=1}^{N_{BC}}$ and $\{1, t^{(i)}\}_{i=1}^{N_{BC}}$.

In [ ]:
def sample_BC(n_BC):
    """
    Generate random sample of the IC points (x, 0) and their corresponding temperature values

    Parameter:
    - n_BC: An integer defining the number of data points sampled from the boundary conditions at each boundary (left and right).

    Returns:
    - X_BC: The coordinates of the boundary conditions points
    - u_BC: The temperature values for the boundary conditions points
    """
    # left boundary (0, t)
    X_left = sample_domain(n_BC)
    X_left[:, 0] = 0
    # right boundary (1, t)
    X_right = sample_domain(n_BC)
    X_right[:, 0] = 1
    
    # Set temperature to zero (cooling)
    u_left = np.zeros((n_BC, 1))
    u_right = np.zeros((n_BC, 1))

    # Combine left and right boundaries to a single dataset
    X_BC = np.vstack([X_left, X_right])
    u_BC = np.vstack([u_left, u_right])
    return X_BC, u_BC

## Generating and Preprocessing Datasets

Since we now have implemented all necessary sampling functions for the IC, BC, and collocation points, we now proceed by sampling and plotting the datasets. **Note:** The collocation points do not have any label attached to them (that is why we call the training semi-supervised).

In [ ]:
from src.plots import plot_datasets

# Specifying number of sampled data points
n_IC = 100
n_BC = 100
n_collocation = 1000

# Generate datasets using sampling functions
X_IC, u_IC = sample_IC(n_IC)
X_BC, u_BC = sample_BC(n_BC)
X_collocation = sample_domain(n_collocation)

# Plot the sampled data points
plot_datasets(X_IC, u_IC, X_BC, u_BC, X_collocation)

As a final preprocessing step, we combine the dataset for IC and BC into a single dataset used for the data loss function. Additionally, we convert all dataset from a `np.array` to `tf.tensor`, necessary for training the PINN using Tensorflow.

In [ ]:
# Concatenate to single training data set
X_train = np.concatenate([X_IC, X_BC], axis=0)
u_train = np.concatenate([u_IC, u_BC], axis=0)

# Convert to tf.Tensor
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
u_train = tf.convert_to_tensor(u_train, dtype=tf.float32)
X_collocation = tf.convert_to_tensor(X_collocation, dtype=tf.float32)

## Defining Training Function and Optimizer

We now implement the `train_step()` function that is used to update the model parameters based on the achieved losses. To obtain the derivative of the loss function with respect to the model parameters, we again use the `tf.GradientTape()` function as implemented in Tensorflow.

One important aspect of PINN training will be used in the function: We have two losses - the data loss $L_D$ encoding the IC/BC, and the physics loss $L_F$ encoding the PDE. Both losses are considered in the PINN training as **Multi-Objective**, and the standard approach for PINN training is formulating a single loss function by the (unweighted) linear combination of both: $L=L_D+L_F$.
 

(**Note:** For a significant runtime speedup, we can use `@tf.function` decorator on the `train_step()` function to convert it into a TensorFlow graph function.)


In [ ]:
@tf.function
def train_step(optimizer, model, X_train, u_train, X_collocation):
    """
    Perform one training step for the PINN model.

    Parameters:
    - optimizer: The optimizer used to minimize the loss.
    - model: The PINN model which approximates the solution u(x,t).
    - X_train: A tensor containing the IC/BC points (x, t).
    - u_train: A tensor containing the known values of u at the IC/BC points.
    - X_collocation: A tensor containing the collocation points (x, t) where the PDE is enforced.

    Returns:
    - loss_u: The loss value for the data loss
    - loss_F: The loss value for the physics loss
    """
    with tf.GradientTape() as tape:
        # Data loss
        loss_data = data_loss(model, X_train, u_train)
        # Physic loss
        loss_physics = physics_loss(model, X_collocation)

        # Linear combination of data and physics loss
        loss_total = loss_data + loss_physics
    # Retrieve derivative of the total loss function w.r.t the model parameters
    grads = tape.gradient(loss_total, model.trainable_variables)
    # Perform a single update step
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss_data, loss_physics

To actually apply the changes to the model parameters in the function implemented above, we have to specify a suitable optimizer. In this case (and as standard for training neural networks) we will use Adam with a default learning rate.

In [ ]:
# Set the learning rate and initialize the optimizer
learning_rate = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate)

## Creating the Physics-Informed Neural Network

We are close to start our model training. The only component missing is the model itself!

To approximate the solution function $u(x,t)$, we use a fully-connected neural network as our model. This model represents a continous function approximator $u_\theta(x,t)$, where $\theta$ represents the trainable model parameters. The input dimension will be automatically set to two (for the input coordinates $x$ and $t$), and the output dimension will be set to one (for the target value $u$). The number of hidden layers, neurons per layer and activation function can be changed at will.

In [ ]:
from src.neural_network import NeuralNetwork

# Initialize the neural network
PINN = NeuralNetwork(n_hidden=2, n_neurons=20, activation='tanh')

## Performing PINN Training

Now we finally start the PINN training with a specified number of training epochs. Within the training loop, a single training step is performed by calling the previously implemented `train_step()` function. Additionally, we use some logging and printing to better understand whether we are effectively training the PINN.

In [ ]:
from time import time

# Number of training epochs
n_epochs = 10000
# Create empty list for recording training logs
log_data, log_physics = [], []
# Start training loop
start_time = time()
for epoch in range(n_epochs):
    # Perform a single training step
    loss_data, loss_physics = train_step(optimizer, PINN, X_train, u_train, X_collocation)
    # Write loss values to log
    log_data.append(float(loss_data))
    log_physics.append(float(loss_physics))
    # Print the loss values 
    if epoch % 1000 == 0:
        print(f'Epoch {epoch:<5} || Loss_data: {loss_data:1.2e} || Loss_physics: {loss_physics:1.2e}')

print(f"===== Finished training in {time()-start_time:.1f} seconds =====")
print(f'Final Value || Loss_data: {loss_data:1.2e} || Loss_physics: {loss_physics:1.2e}')

## Model Evaluation

Since we have now trained our PINN, we can perform a series of model evaluation. We start by looking at the **learning curves** that show the history of loss values plotted against the training epochs.

In [ ]:
from src.plots import plot_learning_curves
plot_learning_curves(log_data, log_physics)

Next, we can use the trained model to make predictions on the solution function over the entire computational domain, which we tried to approximate with our PINN.

In [ ]:
from src.plots import plot_prediction
plot_prediction(PINN)

Finally, we can also check the model's accucary by using a dedicated test set. A common performance measure is the **relative L2 Error** given by $rel. L^2 = ||u_{pred}-u_{true}||/||u_{true}||$. For the test set, we just use a random sample of data points within the computational domain.

In [ ]:
# Obtain a random test dataset
X_test = sample_domain(n_sample=1000)
u_test = get_solution(X_test)

# Make predictions
X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
u_pred = PINN(X_test)

# Determine the relative L2 Error
rel_L2 = np.linalg.norm(u_pred-u_test)/np.linalg.norm(u_test)
print(f"rel. L2 Error: {rel_L2*100:1.3f}%")